------------------------------------------------------------------------------------------------------------------------

This is the results part of the project. In here functions are declared and described. If you want to see them applied and the results, please go to:

 ### __[Results](https://github.com/serbanradulescu/master_thesis/blob/main/project.ipynb)__ |  Code 
------------------------------------------------------------------------------------------------------------------------

In [3]:
import re
import wget
import requests
import pandas as pd
import matplotlib
#import Shapely
import geopandas as gpd
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from requests_html import HTMLSession
from itertools import combinations
from functools import lru_cache
from typing import List, Tuple
#from functions import get_date
from zipfile import ZipFile
from os.path import exists

from time import sleep

ModuleNotFoundError: No module named 'Shapely'

In [ ]:
@lru_cache
def get_links(parameters:tuple[str], time:List[str] = ["1_minute","5_minutes","10_minutes","hourly"]) -> dict:
    dwd_links = {interval:{key: None for key in parameters} for interval in time}
    for interval in time:
        for parameter in parameters:
            url = 'https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/' + str(interval) + '/' + parameter + '/historical/'
            try:
                session = HTMLSession()
                response = session.get(url)
                dwd_links[interval][parameter] = response.html.absolute_links

            except requests.exceptions.RequestException as e:
                print(e)
    return dwd_links

In [ ]:
def get_date(link:str) -> Tuple[int,int]:
    """Returns the years from when the data is available.

    Args:
        link (str): the link that has included the dails for the available data time

    Returns:
        Tuple[int,int]: start time, end time
    """
    m = re.findall('\d{8}', link)
    if m:
        return (int(m[0][:4]),int(m[1][:4]))
    return (0,0)

In [ ]:
def count_datapoints(dwd_links:dict,time,parameter:str, start_year:int, end_year:int) -> int:
    i=0
    for link in dwd_links[time][parameter]:
        try:
            start_interval = int(get_date(link)[0])
            end_interval = int(get_date(link)[1])
            if ((start_interval <= start_year)  & (end_interval >= end_year)):
                i = i+1
        except:
            pass
    return i

In [ ]:
def show_available_data(dwd_links:dict,time, parameters: List[str]):
    data_balance = pd.DataFrame(columns = parameters, index = [str(i) + "'s - present" for i in range(1950,2020,10)])
    for parameter in parameters:
        for i in range(1950,2020,10):
            data_balance[parameter][str(i)+ "'s - present"] = count_datapoints(dwd_links,time,parameter, i, 2020)
    display(data_balance)

In [ ]:
def ids_datapoints(dwd_links:dict,time:str,parameter:str, start_year:int, end_year:int) -> List[str]:
    list = []
    for link in dwd_links[time][parameter]:
        try:
            start_interval = int(get_date(link)[0])
            end_interval = int(get_date(link)[1])
            if ((start_interval <= start_year)  & (end_interval >= end_year)):
                id = re.findall("_\d{5}_",str(link))[0]
                ##print("link")
                list.append(id[1:-1])
        except:
            pass
    return list

parameters =("air_temperature","dew_point", "moisture", "precipitation")
dwd_links = get_links(parameters)

In [ ]:
def common_stations(paramer1:str, parameter2:str, time:str, interval: List[int]):
    print(common_stations)
    print(f"{paramer1} and {parameter2} have the following number of stations that measured in:")
    full_list = []
    for i in interval:
        list1 = ids_datapoints(dwd_links,time,paramer1,i,2020)
        list2 = ids_datapoints(dwd_links,time,parameter2,i,2020)
        list1_as_set = set(list1)
        intersection = list1_as_set.intersection(list2)
        intersection_as_list = List(intersection)
        full_list = full_list + intersection_as_list
        print(str(i) + " - present" + ": " + str(len(intersection_as_list)))        
    return full_list

def simple_common_stations(ids_parameter1: List[int],ids_parameter2: List[int]):
    list1_as_set = set(ids_parameter1)
    intersection = list1_as_set.intersection(ids_parameter2)
    intersection_as_list = list(intersection)
    return intersection_as_list

moisture_air_t = simple_common_stations(ids_datapoints(dwd_links,"hourly","moisture",1950,2020),ids_datapoints(dwd_links,"hourly","air_temperature",1950,2020))
print(len(simple_common_stations(moisture_air_t,ids_datapoints(dwd_links,"hourly","dew_point",1950,2020))))

def complex_common_stations(parameters:List[str],time:str, interval: List[int]):
    ids =[{i:{parameter:ids_datapoints(dwd_links,time,parameter,i,2020)}} for parameter in parameters for i in interval]
    for i in range(0,len(interval)):
        print(ids[i])
    for i in range(2, len(parameters)+1):
        combined = combinations(parameters, i)
        #print(" ".join(str(e) for e in combined) )
        #for j in combined:
            #print(ids)
        #print(set.intersection(*[set(list) for list in p]))       

#complex_common_stations(["moisture", "dew_point", "air_temperature"],"hourly", range(1950,2020,10))


24


In [ ]:
def create_df(parameter,time,start_year,end_year, is_test = True):
    test_count = 0
    if(is_test):
        limit =4
    else:
        limit = 99999
    df = pd.DataFrame()
    for link in dwd_links[time][parameter]:   
        if (test_count <= limit):
            test_count+=1
            filename = str.split(link,"/")[-1]
            
            if(not exists("downloads/" + time + "/" + parameter + "/" + filename)):
                file_zip = wget.download(link,"downloads/"+time+"/" + parameter + "/")
                print("downloading" + filename + "...",end = " ", flush=True)
            else:
                file_zip = "downloads/"+ time+ "/" + parameter + "/" + filename
                print("FOUND "+ filename, end =" ", flush=True)
                sleep(1)
            try:    
                with ZipFile(file_zip) as myzip:
                    for filename in myzip.namelist():
                        if "Metadat" not in filename:
                            with myzip.open(filename) as myfile:
                                this_df = pd.read_csv(myfile, sep =";")
                                
                                df = pd.concat([df,this_df])
            except:
                print("Not able to open:",filename, "Reason: unknown.")
    return df



In [ ]:
def coordinates_stations(ids:List[str], path_info:str ):
    df = pd.read_csv(path_info, header=None, names=["raw"], sep = ";", encoding = "latin1", skiprows=2)
    #df["id"] = df["raw"].apply(lambda x: x[0:5])
    df["id"] = df["raw"].str.extract(r'(\d{5})')
    #df["period"] = df["raw"].str.findall(r'(\d{8})')
    #df["height"] = df["raw"].str.extract(r'(\s+\d{1,3}\s+)')
    df["coord"] = df["raw"].str.findall(r'(\d{1,2}\.{1}\d{4})')
    df[['lat','lon']] = pd.DataFrame(df.coord.tolist(), index= df.index)
    #df["lat"] = float(df["lat"])

    processed_df = df[["id","lat","lon"]]
    if ids == None:
        return processed_df
    else:
        return processed_df[processed_df["id"].isin(ids)]

NameError: name 'List' is not defined

### The following code is written just to email me when the files are downloaded/models are trained

In [ ]:
import smtplib, ssl, os

def mail_me():
    port = 465  # For SSL
    smtp_server = "smtp.gmail.com"
    sender_email = "computersays521@gmail.com"  # Enter your address
    receiver_email = "radulescu_serban_petre@yahoo.com"  # Enter receiver address
    password = open("mailingfile.txt","r").readlines()[0]
    message = """\
    Subject: Hi there

    It seems that what you were working on is done. Or you had some errors. Either way the program is completed. """

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message)